In [2]:
import sys , os 
import numpy as np
import math as m
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

In [21]:
# get datas
mesurePlot="12E/new/validation/profil12eMesure.csv"
gatePlot="12E/new/validation/profil12eGateW.csv"

#create pandas dataFrames
mesurePlotData=pd.read_csv(mesurePlot, sep=';')
gatePlotData=pd.read_csv(gatePlot, sep=',')

## Create data to analyze

In [22]:
#Create numpy ndarray from pandas dataFrames
refX=mesurePlotData['pas']
refY=mesurePlotData['value']
gateX=gatePlotData['pas']
gateY=gatePlotData['value']

# Compute Linear interpolation for simulated values and measured values according to your parameters (here -> 2000 points for one profil)
refX2=np.linspace(refX.min(), refX.max(), 2000)
refY2=np.interp(refX2, refX, refY)
gateX2=np.linspace(gateX.min(), gateX.max(), 2000)
gateY2=np.interp(gateX2, gateX, gateY)

# center meean value normalization
maskFilm=(refX2 > -2) & (refX2 < 2)
maskGate=(gateX2 > -2) & (gateX2 < 2)

refY2=refY2/refY2[maskFilm].mean()
gateY2=gateY2/gateY2[maskGate].mean()

#scaling in x to fit with measurment
gateX2=gateX2+0.5

## Plot with Plotly

In [61]:
# Plot profil or PDD with plotly      
trace1 = go.Scatter(
    x = refX2,
    y = refY2,
    mode = 'lines', 
    name = 'Mesure', 
    line = dict(
        color = ('rgba(214, 39, 40, 100)')
    )
)

trace2 = go.Scatter(
    x = gateX2,
    y = gateY2,
    mode = 'lines', 
    name = 'GATE', 
    line = dict(
        color = ('rgba(255, 127, 17, 100)')
    )
)

layout1= go.Layout(
    xaxis=dict(
        range=[-100, 100],
        title='Off distance axis (mm)',
        titlefont=dict(
            size=20
        )
),

    yaxis=dict(
        title='Relative Dose',
        titlefont=dict(
            size=20)
    )
)


data1 = [trace1, trace2]
fig1 = go.Figure(data=data1, layout=layout1)
py.iplot(fig1, filename='Plot_example', image='png') 

## Field Size calculation

In [62]:
# get mean center value
maximum_mesure=refY2[maskFilm].mean()
maximum_gate=gateY2[maskGate].mean()

# get the 50% of mean center value
half_max_m=0.5* maximum_mesure
half_max_g=0.5 * maximum_gate

# create a boolean mask where profil value sup than the 50% of mean center value
maskFieldSize_m = (refY2 >= half_max_m)
maskFieldSize_g = (gateY2 >= half_max_g)

#apply this mask on distance axis and compute difference between first and last value
field_mes=refX2[maskFieldSize_m][-1]-refX2[maskFieldSize_m][0]
field_gate=gateX2[maskFieldSize_g][-1]-gateX2[maskFieldSize_g][0]

print 'field size MESURE : ', field_mes
print 'field size GATE : ', field_gate

diff_fs=((abs(field_mes-field_gate)/field_mes)*100)
print 'Ecart relatf : ', diff_fs 

field size MESURE :  102.785792896
field size GATE :  102.531265633
Ecart relatf :  0.247628837079


## Penumbra calculation

In [63]:
#Penumbra calculation

#get 80% and 20% of mean center value for measrment 
up_pen_val_ref=0.8 * maximum_mesure
down_pen_val_ref=0.2 * maximum_mesure

#get 80% and 20% of mean center value for  gate 
up_pen_val_gate=0.8 * maximum_gate
down_pen_val_gate=0.2 * maximum_gate


# create a boolean mask to get profile value between  20% and 80% of mean center value (but there is left and right penumbra)
maskPenumbra_mes = (refY2 > down_pen_val_ref) & (refY2 < up_pen_val_ref )
maskPenumbra_gate = (gateY2 > down_pen_val_gate) & (gateY2 < up_pen_val_gate )

#sort the left and right by selecting positive or negative value
maskL_m=refX2[maskPenumbra_mes] < 0
maskR_m= refX2[maskPenumbra_mes] > 0

maskL_g=refX2[maskPenumbra_gate] < 0
maskR_g=refX2[maskPenumbra_gate] > 0

#apply the last two boolean mask on distance axis and compute the difference between the last and the first 
pen_L_mes = refX2[maskPenumbra_mes][maskL_m][-1]-refX2[maskPenumbra_mes][maskL_m][0]
pen_R_mes = refX2[maskPenumbra_mes][maskR_m][-1]-refX2[maskPenumbra_mes][maskR_m][0]

pen_L_gate = gateX2[maskPenumbra_gate][maskL_g][-1]-gateX2[maskPenumbra_gate][maskL_g][0]
pen_R_gate = gateX2[maskPenumbra_gate][maskR_g][-1]-gateX2[maskPenumbra_gate][maskR_g][0]


print 'penumbra left (ref, gate ) : ' , pen_L_mes, pen_L_gate
diff_pl=((abs(pen_L_mes-pen_L_gate)/pen_L_mes)*100)
print 'écart relatif : ', diff_pl
print
print 'penumbra right (ref, gate ) : ' , pen_R_mes, pen_R_gate
diff_pr=((abs(pen_R_mes-pen_R_gate)/pen_R_mes)*100)
print 'écart relatif : ', diff_pr

penumbra left (ref, gate ) :  9.38869434717 8.8044022011
écart relatif :  6.22335890878

penumbra right (ref, gate ) :  9.06213106553 8.8044022011
écart relatif :  2.84402049108


## Flatness Study

In [23]:
#Flatness  study


#bolean mask for value more than 80% of D max
maskFlat_mes=(refX2 > -40) & (refX2 < 40)
maskFlat_gate=(gateX2 > -40) & (gateX2 < 40)

m_m = refY2[maskFlat_mes].min()
m_g = gateY2[maskFlat_gate].min()
M_m = refY2[maskFlat_mes].max()
M_g = gateY2[maskFlat_gate].max()

#print 'min (ref et gate) ', m_m , m_g
#print 'max (ref et gate) ', M_m , M_g

F_m = ((M_m - m_m)/(M_m + m_m)) * 100
F_g = ((M_g - m_g)/(M_g + m_g)) * 100

print 'Flatness MESURE : ', F_m
print 'Flatness GATE : ',  F_g
diff_f=((abs(F_m-F_g)/F_m)*100)
print 'Ecart relatif : ' , diff_f

Flatness MESURE :  3.00381491246
Flatness GATE :  3.04940997273
Ecart relatif :  1.51790511702
